In [1]:
import sqlite3
import pandas as pd


conn = sqlite3.connect(r"C:\Users\foxgh\OneDrive\Área de Trabalho\SQL Playground\Script\data.db")
cursor = conn.cursor()

In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("Tabelas no banco de dados:", tabelas)


Tabelas no banco de dados: [('vendas',), ('estoque',), ('estoque_detalhado',)]


In [3]:
query = "select * from estoque"

df = pd.read_sql_query(query, conn)

# conn.close()

df.head()

,ID,Produto,Categoria,Estoque_Inicial,Estoque_Atual,Fornecedor,Data_Reabastecimento
0,1,Produto_1,Móveis,264,222,Fornecedor_7,2023-01-01
1,2,Produto_2,Brinquedos,445,432,Fornecedor_10,2023-01-16
2,3,Produto_3,Móveis,213,178,Fornecedor_8,2023-01-31
3,4,Produto_4,Alimentos,407,375,Fornecedor_4,2023-02-15
4,5,Produto_5,Móveis,431,408,Fornecedor_8,2023-03-02


In [4]:
query = """ 
    select 
        produto,
        sum(estoque_atual) as estoque,
        sum(estoque_inicial) - sum(estoque_atual) as vendas,
        case when sum(estoque_inicial) < 100 then 'Alerta' else 'Ok' end as status
    from estoque
    group by 1
"""

df = pd.read_sql_query(query, conn)

df

,Produto,estoque,vendas,status
0,Produto_1,222,42,Ok
1,Produto_10,107,32,Ok
2,Produto_100,305,49,Ok
3,Produto_101,272,37,Ok
4,Produto_102,118,15,Ok
...,...,...,...,...
195,Produto_95,355,33,Ok
196,Produto_96,188,29,Ok
197,Produto_97,128,16,Ok
198,Produto_98,27,28,Alerta


In [ ]:
query = """ 
    select 
        id,
        max(ultima_compra) as ultima_compra,
        sum(quantidade_recebida) as quantidade_recebida,
        cast(julianday(data_proxima_entrega) - julianday(ultima_compra) as integer) as dias_entrega,
        status_estoque as status  
    from estoque_detalhado
    group by 1,5
"""

df = pd.read_sql_query(query, conn)

df

,ID,ultima_compra,quantidade_recebida,dias_entrega,status
0,1,2023-01-01,143,181,Esgotado
1,2,2023-01-16,149,186,Disponível
2,3,2023-01-31,227,191,Crítico
3,4,2023-02-15,389,196,Esgotado
4,5,2023-03-02,111,201,Disponível
...,...,...,...,...,...
195,196,2031-01-04,402,1156,Disponível
196,197,2031-01-19,117,1161,Baixo
197,198,2031-02-03,378,1166,Crítico
198,199,2031-02-18,419,1171,Esgotado


In [69]:
query = """ 

with t1 as (
    select 
        id,
        produto,
        sum(estoque_atual) as estoque,
        sum(estoque_inicial) - sum(estoque_atual) as vendas,
        case when sum(estoque_inicial) < 100 then 'Alerta' else 'Ok' end as status
    from estoque
    group by 1
),
    
t2 as (
    select 
        id,
        max(ultima_compra) as ultima_compra,
        sum(quantidade_recebida) as quantidade_recebida,
        cast(julianday(data_proxima_entrega) - julianday(ultima_compra) as integer) as dias_entrega,
        status_estoque as status,
        temperatura_armazenamento
    from estoque_detalhado
    group by 1,5
)    

select 
    t1.*,
    t2.ultima_compra,
    t2.dias_entrega,
    t2.quantidade_recebida,
    t2.temperatura_armazenamento
    
from t1 
left join t2 on t1.id = t2.id

    
"""

df = pd.read_sql_query(query, conn)

df

,id,produto,estoque,vendas,status,ultima_compra,dias_entrega,quantidade_recebida,temperatura_armazenamento
0,1,Produto_1,222,42,Ok,2023-01-01,181,143,Refrigerado
1,2,Produto_2,432,13,Ok,2023-01-16,186,149,Temperatura Ambiente
2,3,Produto_3,178,35,Ok,2023-01-31,191,227,Temperatura Ambiente
3,4,Produto_4,375,32,Ok,2023-02-15,196,389,Temperatura Ambiente
4,5,Produto_5,408,23,Ok,2023-03-02,201,111,Refrigerado
...,...,...,...,...,...,...,...,...,...
195,196,Produto_196,365,10,Ok,2031-01-04,1156,402,Temperatura Ambiente
196,197,Produto_197,230,27,Ok,2031-01-19,1161,117,Seco
197,198,Produto_198,37,45,Alerta,2031-02-03,1166,378,Seco
198,199,Produto_199,314,15,Ok,2031-02-18,1171,419,Seco


In [ ]:
query = """ 
    select 
        fornecedor,
        count(distinct produto) as total_produtos,
        sum(estoque_atual) as total_estoque,
        end as classificacao
    
    from estoque
    group by fornecedor
    order by total_estoque desc

"""

df = pd.read_sql_query(query, conn)

df

,Fornecedor,total_produtos,total_estoque,classificacao
0,Fornecedor_2,28,7530,Alto
1,Fornecedor_9,24,6546,Alto
2,Fornecedor_7,23,5972,Alto
3,Fornecedor_10,25,5432,Alto
4,Fornecedor_5,19,5174,Alto
5,Fornecedor_8,18,4793,Alto
6,Fornecedor_4,18,4606,Alto
7,Fornecedor_1,15,4103,Alto
8,Fornecedor_3,15,4063,Alto
9,Fornecedor_6,15,3852,Alto
